# 1. Import Dependencies

In [ ]:
!pip install opencv-python
#!pip install Cython>=0.29.12

In [18]:
# Import opencv
import cv2 

# Import uuid
import uuid

# Import Operating System
import os

# Import time
import time

# 2. Define Images to Collect

In [4]:
labels = ['mask', 'no-mask']
number_imgs = 5

# 3. Setup Folders 

In [ ]:
pwd

In [6]:
IMAGES_PATH = os.path.join('images', 'collectedimages')

In [89]:
print(IMAGES_PATH)

images\collectedimages


In [8]:
os.name

'nt'

In [9]:
if not os.path.exists(IMAGES_PATH):
    if os.name == 'posix':      # for colab
        !mkdir -p {IMAGES_PATH}
    if os.name == 'nt':         # for windows
         !mkdir {IMAGES_PATH}
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        !mkdir {path}

# 4. Capture Images

In [12]:
for label in labels:
    #CONNECTS TO WEBCAM OR CAPTURE DEVICE
    cap = cv2.VideoCapture(0)     
    print('Collecting images for {}'.format(label))
    time.sleep(15)
    for imgnum in range(number_imgs):
        print('Collecting image {}'.format(imgnum))
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH,label,label+'.'+'{}.jpg'.format(str(uuid.uuid1())))
        cv2.imwrite(imgname, frame)
        cv2.imshow('frame', frame)
        time.sleep(5)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

# 5. Image Labelling

### iNSTALLATION INSTRUCTION
### https://github.com/heartexlabs/labelImg

Windows + Anaconda

- Download and install Anaconda (Python 3+)
- Open the Anaconda Prompt and go to the 'labelImg' directory

- conda install pyqt=5
- conda install -c anaconda lxml
- pyrcc5 -o libs/resources.py resources.qrc
- python labelImg.py
- python labelImg.py [IMAGE_PATH] [PRE-DEFINED CLASS FILE]

In [9]:
#!pip install --upgrade pyqt5 lxml

In [27]:
# give any path where you would like to install 'LabelImg'
#LABELIMG_PATH = os.path.join(os.getcwd(), 'labelimg')

In [ ]:
#LABELIMG_PATH

In [ ]:
# if not os.path.exists(LABELIMG_PATH):
#     !mkdir {LABELIMG_PATH}
#     !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [13]:
# if os.name == 'posix':
#     !make qt5py3
# if os.name =='nt':
#     !cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
#!cd {LABELIMG_PATH} && python labelImg.py

# 6. Generate CSV 

In [109]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [112]:
for folder in ['collectedimages']:
    images_Folder_path = os.path.join(os.getcwd(), 'images/' + 'collectedimages')
    xml_df = xml_to_csv(images_Folder_path)
    xml_df.to_csv((images_Folder_path + '/' + '_labels.csv'), index=None)
    print('Successfully converted xml to csv.')

Successfully converted xml to csv.


In [105]:
#pwd

'C:\\Users\\maz\\dev\\Projects_\\MaskDetection'

In [108]:
#os.path.join(os.getcwd(), 'images/' + 'collectedimages')

'C:\\Users\\maz\\dev\\Projects_\\MaskDetection\\images/collectedimages'

In [ ]:
df = pd.read_csv(os.path.join(os.getcwd(), 'images', 'collectedimages', '_labels.csv'))
df

In [114]:
df['class'].value_counts()

mask       106
no-mask    106
Name: class, dtype: int64

In [ ]:
sns.set (style = 'darkgrid')
ax = sns.countplot(x = 'class', data = df)

# 7. Move them into a Training and Testing Partition

# OPTIONAL - 8. Compress them for Colab Training

In [ ]:
import os

In [117]:
TRAIN_PATH = os.path.join('images', 'train')
TEST_PATH = os.path.join('images', 'test')
ARCHIVE_PATH = os.path.join('images', 'archive.tar.gz')

In [ ]:
!tar -czf {ARCHIVE_PATH} {TRAIN_PATH} {TEST_PATH}